# Statistical Testing

## Statistical Hypothesis

In section [Parametric Distributions](../stats_03_parametric_distributions.ipynb) we discussed how several parametric models can "sort of fit" the empirical distribution. Now suppose that we want to limit the choices and we only want to compare two specific models and decide which one is "more probable". The two models can follow the same family of distributions but with two sets of parameters, for example, normal with two different means, or we can compare the normal distribution to logistic with fixed parameters. Based on these two options, we can design two hypotheses: 

$\mathcal{H_0}: X \sim p_0(\theta_0),$

$\mathcal{H_1}: X \sim p_1(\theta_1),$

where $p_0(\theta_0)$ and $p_1(\theta_1)$ are the two different distributions with their corrosponding parameters.

## Hypothesis Test

We would like to come up with a decision strategy based on which to determine if the data supports a certain hypothesis. A **hypothesis test** uses a **test statistic** which is a function of the sample, based on the outcome of which we can distinguish between the two hypotheses. If we assume that the null hypothesis is true, and calculate the distribution of the test statistics under the null hypothesis, we would like to reject the null hypothesis if we observe that the probability of the observed test statistic is low. The region for which the null hypothesis is rejected is called a **critical region**.

How can we come up with such a test statistic and the cutoff for rejection? We need to accepts that we will make some errors.


|Null hypothesis is | TRUE | FALSE|
|-------------------|------|------|
|Not Rejected| Correct| Type I Error |
|Rejected| Type II Error| Correct |


The probability of Type I Error is called a **significance** level and usually denoted by $\alpha$.

The probability of Type II Error is denoted by $\beta$, and the **power** of the test is $1-\beta$.

To design a "good test" one needs to juggle both errors.

### Likelihood Ratio Test

Suppose $X$ was a discrete variable. Then the probability of observing $x$ under each hypothesis is equal to the likelihood function of each model evaluated at $x$. So a measure of which hypothesis is "more likely" is the ratio of the likelihoods and we can design a test statistic based on it:

$$\Lambda(X) = \frac{\mathcal{L(\theta_0|X)}}{\mathcal{L(\theta_1|X)}}$$

and design a test, with a significance level $\alpha$:

Reject $H_0$:$\Lambda(x)\le c$,

Do not reject $H_1$: $\Lambda(x)> c$,

where $c$ is chosen so that $P(\Lambda(X)\le c|\mathcal{H_0}) = \alpha$.




Such a test can be specified also for continuous distributions. It turns out that *the likelihood ratio test is the most powerful test among the tests of significance level $\alpha$*! Many of the popular tests in statistics are likelihood ratio tests.



## Ship Noise Detection Example

We want to determine whether ship noise is present in the environment. If there no ship noise, we assume that the distribution of the ambient noise follows a Gaussian distribution. If there is ship noise, the distribution will be skewed. The alternative hypothesis is that the observations are from Gamma distribution. 


$\mathcal{H_0}: X \sim \mathcal{N}(\mu_0, \sigma^2)$

$\mathcal{H_1}: X \sim \mathcal{\Gamma}(\alpha, scale, loc)$


In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&confirm=t&id=1466snzjsXPVTlKnzkkCkdOgwoO5Zvutq' -O background.wav

--2025-03-14 10:57:06--  https://docs.google.com/uc?export=download&confirm=t&id=1466snzjsXPVTlKnzkkCkdOgwoO5Zvutq
Resolving docs.google.com (docs.google.com)... 172.217.14.206, 2607:f8b0:400a:801::200e
Connecting to docs.google.com (docs.google.com)|172.217.14.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1466snzjsXPVTlKnzkkCkdOgwoO5Zvutq&export=download [following]
--2025-03-14 10:57:07--  https://drive.usercontent.google.com/download?id=1466snzjsXPVTlKnzkkCkdOgwoO5Zvutq&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.33.97, 2607:f8b0:400a:807::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.33.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5759054 (5.5M) [audio/wav]
Saving to: ‘background.wav’

background.wav      100%[===================>]   5.49M  --.-KB/s    in 0.07s   

2025

In [3]:
from scipy.io import wavfile
# reading background data
bg_samplerate, bg_signal = wavfile.read('background.wav')
import numpy as np
# first we split small intervals of 0.1s
bg_signal_split = np.split(bg_signal[:(len(bg_signal)-len(bg_signal)%bg_samplerate)], len(bg_signal[:(len(bg_signal)-len(bg_signal)%bg_samplerate)])/bg_samplerate*10)
# we calculate RMS for each interval
RMS_split = [np.sqrt(np.mean(np.square(group.astype('float')))) for group in bg_signal_split]

We will follow the following procedure:

1. Set a significance level
2. Calculate the test statistic
3. Determine the critical region based on alpha and the test statistic
4. Make decision

**1. Significance level $\alpha = 0.01$**

Despite some acceptance within specific fields of what a reasonable significance level is, it is important to interpret what it means in terms of the context. In this case, it means that the probability of the scientific problem. In this case, it means if we perform the experiment repeatedly, we will on average wrongly predict ship noise when it is not present $1\%$ of the time. If we would like to detect times when ships were present in a protected zone in which they are not supposed to be present, and we would like to use the detections to notify officials if a ship is present, we would prefer to be really certain that there is a violation before do that. If we are studying the effect of noise on a another process, we could possibly incorporate that error in the follow up analysis, and it is less important about the specific choice, as long as it is small.

**2. Likelihood-ratio test statistic**


In [4]:
import scipy.stats as stats

mean = np.mean(np.log10(RMS_split))
std = np.std(np.log10(RMS_split))
# calculate skewness
sk = stats.skew(np.log10(RMS_split), bias=True)

In [5]:
# estimate parameters
a = 4/(sk**2)
# offset = mean - std*np.sqrt(a)
offset = 1.9
scale = std**2/(mean - offset)

scale = 0.06
a = 3



In [6]:
gaussian_density = stats.norm.pdf(np.log10(RMS_split), loc=mean, scale=std)
gamma_density = stats.gamma.pdf(np.log10(RMS_split), a=a, scale=scale, loc=offset)

In [7]:
print("The guassian likelihood is " +str(np.prod(gaussian_density)))
print("The gamma likelihood is " +str(np.prod(gamma_density)))

The guassian likelihood is 1.2385214402124535e+250
The gamma likelihood is 2.2712715048394933e+259


We can hypothesize that the gamma model is more probable. But we need to test this properly.

We compute the likelihood ratio test statistic:

In [8]:
L = np.prod(gaussian_density)/np.prod(gamma_density)

In [9]:
print(L)

5.452987181732717e-10


**3. Identify critical region**

We would like to determine the threshold $c$ for which the significance level of the test is $\alpha$. Note this can be done before actually observing the sample. What we need for that is to determine the distribution of the likelihood ratio under null hypothesis distribution: in this case the Gaussian distribution with the pre-specified parameters (in this case we extracted them from the sample but we will treat them as known). 

$P(\Lambda(X)\le c)$ = $0.01$, where $X \sim \mathcal{N}(\mu, \sigma)$

Instead of trying to calculate it analytically, we will simulate a large sample from the Gaussian distribution, evaluate the likelihood ratio on it, and calculate its first percentile.

In [10]:
# simulate sample from the null hypothesis
x = stats.norm.rvs(mean, std, size=10000)

In [11]:
# evaluate the likelihood ratio at each point

print(offset)

gaussian_density = stats.norm.pdf(x, loc=mean, scale=std)
gamma_density = stats.gamma.pdf(x, a=a, scale=scale, loc=0)
L = np.prod(gaussian_density)/np.prod(gamma_density)
print(np.prod(gaussian_density))
print(np.prod(gamma_density))
print(sum(gamma_density==0))
print(L)

1.9
inf
0.0
0
inf


/home/codespace/.local/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [12]:
offset = min(np.log10(RMS_split)) - 0.1

In [13]:
offset

np.float64(1.817561646822914)

In [14]:
min(gaussian_density)

np.float64(0.000245354920255676)

Note that even if one sample point is outside of the range of the model's probability density function, the evaluation at that point will be zero, and due to the independence assumption the whole likelihood will be zero. That in a way makes sense: if we observe a point which has probability zero for a specific model, then it means that this model has not produced this observation. In practice, there will be always out

In [15]:
a = 2
skewnormal_density = stats.skewnorm.pdf(x, a=a, scale=std, loc=mean-0.08)

In [16]:
L = np.prod(gaussian_density)/np.prod(skewnormal_density)
print(np.prod(gaussian_density))
print(np.prod(skewnormal_density))
print(L)

inf
inf
nan


/tmp/ipykernel_1918/437195330.py:1: RuntimeWarning: invalid value encountered in scalar divide
  L = np.prod(gaussian_density)/np.prod(skewnormal_density)


In [17]:
logL = np.sum(np.log(gaussian_density)) - np.sum(np.log(skewnormal_density))
print(np.sum(np.log(gaussian_density)))
print(np.sum(np.log(skewnormal_density)))
print(logL)

9880.046020029358
7801.742789110331
2078.3032309190276


In [24]:
def evaluateLogL():
  x = stats.norm.rvs(mean, std, size=len(RMS_split))
  gaussian_density = stats.norm.pdf(x, scale=std, loc=mean)
  skewnormal_density = stats.skewnorm.pdf(x, a=a, scale=std, loc=mean-0.08)
  logL = np.sum(np.log(gaussian_density)) - np.sum(np.log(skewnormal_density))
  return(logL)


In [27]:
# calculate 1 percentile
np.percentile([evaluateLogL() for i in range(1000)], 0.01)

np.float64(62.44770745124872)

In [29]:
gaussian_density = stats.norm.pdf(np.log10(RMS_split), loc=mean, scale=std)
skewnormal_density = stats.skewnorm.pdf(np.log10(RMS_split), a=a, scale=std, loc=mean)
logL = np.sum(np.log(gaussian_density)) - np.sum(np.log(skewnormal_density))
print(np.sum(np.log(gaussian_density)))
print(np.sum(np.log(skewnormal_density)))
print(logL)

575.860191529744
48.756884014672494
527.1033075150715


# Extra Topics
* P-Value
* Example with likelihood ratio and signal detection
* Widgets about significance level and power
* Plot of two distributions